In [9]:
%pip install tqdm

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 600.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\tqdm.exe' -> 'c:\\Python312\\Scripts\\tqdm.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
import time
from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Variables List + Header API

In [2]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
my_game_name = os.getenv("my_game_name")
my_tag_line = os.getenv("my_tag_line")

In [3]:
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

In [4]:
api_url = "https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"

resp = requests.get(api_url, headers=headers)
response = resp.json()
entries = response.get('entries', [])
entries = entries[:4]
test_df = pd.DataFrame([{"summonerId" : entry["summonerId"]} for entry in entries])
test_df.to_csv("test.csv", index = False)

In [4]:
apex_tiers = ['challengerleagues', 'grandmasterleagues', 'masterleagues']
#divisions = ['DIAMOND', "EMERALD"]
divisions = ['DIAMOND']
tiers = ["I", "II", "III", "IV"]
#tiers = ["I"]

CURRENT_PATCH = "15.3"

In [ ]:
def get_summoner_ids_from_api(api_url, req_type):
    summoner_ids = []
    api_url = api_url #dont think this needs to be here lol
    try:
        resp = requests.get(api_url, headers=headers )  #Insert API link here
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")   
    response = resp.json()
    if req_type == "apex":
        entries = response['entries']
        for entry in entries:
            summoner_ids.append(entry['summonerId'])
    if req_type == "regular":
        for entry in response:
            summoner_ids.append(entry['summonerId'])
    return summoner_ids


def get_sums_for_apex_leagues():
    apex_summoner_ids = []
    for apex_league in apex_tiers:
        print(apex_league)
        api_url = f"https://na1.api.riotgames.com/lol/league/v4/{apex_league}/by-queue/RANKED_SOLO_5x5"
        summoner_ids = get_summoner_ids_from_api(api_url, "apex")
        apex_summoner_ids += summoner_ids
    return apex_summoner_ids

def get_sums_for_reg_divisions(start_time):
    inner_start_time = start_time
    total_loops = 3 #starts at 3 since there should be 3 apex tier calls before this
    reg_division_ids = []
    for division in divisions:
        print(division)
        for tier in tiers:
            print(tier)
            page = 1
            more_pages=True
            while more_pages:
                if total_loops == 100: #if we have made 100 requests
                    dif = time.time()-inner_start_time
                    if dif< 120: #if it has not been 120 seconds
                        print(f'sleeping for {dif}')
                        time.sleep(dif)
                        inner_start_time = time.time() #reset start time
                        total_loops = 0 #reset number of loops as well
                api_url = f'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page={page}'
                output_list = get_summoner_ids_from_api(api_url, "regular")
                if output_list:
                    print(page)
                    page += 1
                    reg_division_ids += output_list
                else:
                    more_pages = False
                total_loops += 1 #keeping track of number of api calls for rate limiting purposes
    return reg_division_ids

def get_all_wanted_sums():
    all_sum_ids = []
    start = time.time()
    all_sum_ids += get_sums_for_apex_leagues()
    all_sum_ids += get_sums_for_reg_divisions(start)
    return all_sum_ids


#could later make this a dataframe the whole time, and add the rank information along with the summoner ID
# schema = {'summonerId': 'str', 'Division': 'str', 'Tier': 'str'}
# df = pd.DataFrame(columns=schema.keys()).astype(schema)

all_wanted_sum_ids = get_all_wanted_sums()
#print(all_wanted_sum_ids)
print(len(all_wanted_sum_ids))
all_wanted_sum_ids = list(set(all_wanted_sum_ids))
print(len(all_wanted_sum_ids)) #see if it dropped any duplicates

#took ~4 mins to do diamond +


In [ ]:
sums_df = pd.DataFrame({"summonerId": all_wanted_sum_ids})
sums_df

In [ ]:
sums_df.to_csv("all_wanted_summoner_ids.csv")

# Function to read csv above and populate it (and the dataframe) with puuids in a new column

In [6]:
def get_puuid(csv_only_summ_ids):
    # Assigning csv file here as a placeholder, can just include it in the argument when using this fucntion
    df_only_summ_ids = pd.read_csv(csv_only_summ_ids, index_col=False)
    # Loop to iterate through each row
    for index, row in df_only_summ_ids.iterrows():
            summoner_id = row["summonerId"]
            # Variable to insert summ_id in API url that fetches account data
            api_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
            try:
                resp = requests.get(api_url, headers=headers)  #Insert API link here
                resp.raise_for_status()
                summoner_info = resp.json()
                # Create a column in the data frame called puuids and populate it with the puuid for each summoner id
                df_only_summ_ids.at[index, "puuid"] = summoner_info.get("puuid", "Not Found") # Store puuid in df, output Not Found if KeyError
            # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
            except requests.RequestException as e:
                print(f"Error fetching puuid for {summoner_id}: {e}")
    # Update csv file with the new column
    df_only_summ_ids.to_csv(csv_only_summ_ids, index = False)
# Calling function to add new info to existing csv instead of creating new one (using test csv from above in this case)

In [ ]:
get_puuid("all_wanted_summoner_ids.csv")
# get_puuid("test.csv") -> used with the test tiny df


# Function to get match ids using puuids from Dataframe

In [9]:
def get_match_ids(csv_summids_and_puuids, current_patch):
    # Read csv file with summoner ids and puuids 
    df_summoners = pd.read_csv(csv_summids_and_puuids)
    # Create a new DataFrame to store match data
    df_all_matches = pd.DataFrame(columns = ["summonerId", "puuid", "matchId", "matchData"])
    # Variable for number of matches, will be defaulted to max (100) in actual code, small number used for testing
    number_of_matches = 5
    # Iterate through each row in df_summoners
    for _, row in df_summoners.iterrows():
        summoner_id = row["summonerId"]
        puuid = row["puuid"]
        # Fetch match ids for each puuid
        match_history_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={number_of_matches}"

        try:
            resp = requests.get(match_history_api_url, headers=headers)
            resp.raise_for_status()
            match_ids = resp.json()
            # Iterate through each match id and fetch match data
            for match_id in match_ids:
                match_data_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
                response = requests.get(match_data_api_url, headers=headers)
                match_data = response.json()
                # Variable to check for patch/version
                game_patch = (match_data["info"]["gameVersion"].split("."))[0] + "." + (match_data["info"]["gameVersion"].split("."))[1]
                if game_patch == current_patch:
                    # Store match data in JSON file along with summoner id and puuid
                    df_mini = pd.DataFrame({
                        "summonerId" : [summoner_id],
                        "puuid" : [puuid],
                        "matchId" : [match_id],
                        "matchData": [json.dumps(match_data)]
                        })
                    # Append to main DataFrame
                    df_all_matches = pd.concat([df_all_matches, df_mini], ignore_index = True)
                else:
                    continue
        # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
        except requests.RequestException as e:
            print(f"Error fetching matches for {puuid}: {e}")
    df_all_matches = df_all_matches.drop_duplicates("matchId")
    # Save the expanded DataFrame
    df_all_matches.to_csv("all_match_data.csv", index = False)
    print(df_all_matches.nunique())
    return df_all_matches

In [ ]:
# Running get_match function, argument 1 = csv file, argument 2 = current patch
get_match_ids("all_wanted_summoner_ids.csv", CURRENT_PATCH)
# get_match_ids("test.csv", CURRENT_PATCH) -> used with sammple test df for testing

summonerId     4
puuid          4
matchId       20
matchData     20
dtype: int64


,summonerId,puuid,matchId,matchData
0,CNjaL14aTWqJ115uHq5HDsicaFinCyUf1tijNkrwyxwjBfo,edUC7mJ4JZLKNfGVUgBXPOZczXmO28Qddd3_86ogdCCMGD...,NA1_5231746738,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
1,CNjaL14aTWqJ115uHq5HDsicaFinCyUf1tijNkrwyxwjBfo,edUC7mJ4JZLKNfGVUgBXPOZczXmO28Qddd3_86ogdCCMGD...,NA1_5231711705,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
2,CNjaL14aTWqJ115uHq5HDsicaFinCyUf1tijNkrwyxwjBfo,edUC7mJ4JZLKNfGVUgBXPOZczXmO28Qddd3_86ogdCCMGD...,NA1_5231688648,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
3,CNjaL14aTWqJ115uHq5HDsicaFinCyUf1tijNkrwyxwjBfo,edUC7mJ4JZLKNfGVUgBXPOZczXmO28Qddd3_86ogdCCMGD...,NA1_5231669436,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
4,CNjaL14aTWqJ115uHq5HDsicaFinCyUf1tijNkrwyxwjBfo,edUC7mJ4JZLKNfGVUgBXPOZczXmO28Qddd3_86ogdCCMGD...,NA1_5231656392,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
5,91Ss1qJAcTaZTB9TZUhJVyiN8xyYGskeuJ7CZQXcv-N5N0w,2sReskMwzDAGB_N0wh_RAUQ2s1c5ml9wx1yntl4EjFPzV-...,NA1_5231365533,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
6,91Ss1qJAcTaZTB9TZUhJVyiN8xyYGskeuJ7CZQXcv-N5N0w,2sReskMwzDAGB_N0wh_RAUQ2s1c5ml9wx1yntl4EjFPzV-...,NA1_5231343670,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
7,91Ss1qJAcTaZTB9TZUhJVyiN8xyYGskeuJ7CZQXcv-N5N0w,2sReskMwzDAGB_N0wh_RAUQ2s1c5ml9wx1yntl4EjFPzV-...,NA1_5231332019,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
8,91Ss1qJAcTaZTB9TZUhJVyiN8xyYGskeuJ7CZQXcv-N5N0w,2sReskMwzDAGB_N0wh_RAUQ2s1c5ml9wx1yntl4EjFPzV-...,NA1_5231170995,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
9,91Ss1qJAcTaZTB9TZUhJVyiN8xyYGskeuJ7CZQXcv-N5N0w,2sReskMwzDAGB_N0wh_RAUQ2s1c5ml9wx1yntl4EjFPzV-...,NA1_5231132929,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."


Aggregate all data at the champion level:
Created 3 functions:
1. create_match_df(): 
    Takes a dataframe as input (for now it takes a csv file but can be easily changed to a df if needed). It then accesses the matchData column of that df and calls the create_df_participants() and create_df_teamdata() functions using the df it took from input (assigned to df_all_matches) as the argument for both functions. 
    - Next step would be to aggregate the data obtained from both helper functions at the champion level

2. create_df_participants():
    Takes the "matchData" column of df inputted to create_match_df() as input. It then extracts the value from the participant key (inside the info dictionary) and creates a new df where each participant in each game is a row and the columns are the data points from the matchdata API endpoint. This function flattens out the entire dictionary (so the keys from sub-dictionaries like "challenges" are columns). This also includes matchIds.

3. create_df_teamdata():  
    Takes the "matchData" column of df inputted to create_match_df() as input. It then extracts the value from the teams key (inside the info dictionary) and creates a new df where each team in each game is a row and the columns are the data points from the matchdata API endpoint (bans, objectives, etc.).
    IMPORTANT: The bans information is still nested in a list of dictionaries, where each dictionary is a ban.



In [21]:
# Function to create a list with all match data
def create_match_df(dataframe_csv: str) -> list[dict]:
    df_all_matches = pd.read_csv(dataframe_csv)
    df_all_matches["matchData"] = df_all_matches["matchData"].apply(json.loads)
    df_participants = create_df_participants(df_all_matches)
    df_teams = create_df_teamdata(df_all_matches)
    return df_participants, df_teams  
    
# Create a function to create a DataFrame from the participants column (i.e. a list of dictionaries)
def create_df_participants(df_all_matches: pd.DataFrame) -> pd.DataFrame:
    # Temporary code to filter by ranked games only (queuId = 420), will be removed once we include this logic where we pull match ids
    df_all_matches["queueId"] = df_all_matches["matchData"].apply(lambda m: m["info"]["queueId"])
    df_all_matches = df_all_matches[df_all_matches["queueId"] == 420].copy(deep = True)
    # Actual function logic below remains unchanged
    df_all_matches["participants"] = df_all_matches["matchData"].apply(lambda m: m["info"]["participants"])
    df_exploded = df_all_matches.explode("participants").reset_index(drop=True)
    df_participants = pd.json_normalize(df_exploded["participants"], sep = "_")
    df_participants["matchId"] = df_exploded["matchData"].apply(lambda m: m["metadata"]["matchId"])
    return df_participants

# Create a function to create a DataFrame with data from each team (each row is a team), like bans and objectives
def create_df_teamdata(df_all_matches: pd.DataFrame) -> pd.DataFrame:
    # Temporary code to filter by ranked games only (queuId = 420), will be removed once we include this logic where we pull match ids
    df_all_matches["queueId"] = df_all_matches["matchData"].apply(lambda m: m["info"]["queueId"])
    df_all_matches = df_all_matches[df_all_matches["queueId"] == 420].copy(deep = True)
    # Actual function logic below remains unchanged
    df_all_matches["teams"] = df_all_matches["matchData"].apply(lambda m: m["info"]["teams"])
    df_exploded = df_all_matches.explode("teams").reset_index(drop=True)
    df_teams = pd.json_normalize(df_exploded["teams"], sep = "_")
    df_teams["matchId"] = df_exploded["matchData"].apply(lambda m: m["metadata"]["matchId"])
    return df_teams

In [22]:
df_participants, df_teams = create_match_df("match_data.csv")
# Took 48s

In [15]:
df_participants.dtypes

PlayerScore0                                 int64
PlayerScore1                                 int64
PlayerScore10                                int64
PlayerScore11                                int64
PlayerScore2                                 int64
                                            ...   
challenges_earliestBaron                   float64
challenges_fasterSupportQuestCompletion    float64
challenges_fastestLegendary                float64
challenges_hadAfkTeammate                  float64
matchId                                     object
Length: 299, dtype: object

In [16]:
df_teams.dtypes

bans                           object
teamId                          int64
win                              bool
objectives_atakhan_first         bool
objectives_atakhan_kills        int64
objectives_baron_first           bool
objectives_baron_kills          int64
objectives_champion_first        bool
objectives_champion_kills       int64
objectives_dragon_first          bool
objectives_dragon_kills         int64
objectives_horde_first           bool
objectives_horde_kills          int64
objectives_inhibitor_first       bool
objectives_inhibitor_kills      int64
objectives_riftHerald_first      bool
objectives_riftHerald_kills     int64
objectives_tower_first           bool
objectives_tower_kills          int64
matchId                        object
dtype: object

In [ ]:
def aggregate_champion_data(df_participants: pd.DataFrame, df_teams: pd.DataFrame) -> pd.DataFrame:
    # Create a teamId column in participants based on participantId (1-5 = 100, 6-10 = 200)
    df_participants['teamId'] = df_participants['participantId'].apply(lambda x: 100 if x <= 5 else 200)
    
    # Merge participants with their team data
    merged_df = df_participants.merge(
        df_teams,
        on=['matchId', 'teamId'],
        how='left'
    )
    
    # Aggregate by champion
    champion_stats = merged_df.groupby(['championId', 'championName']).agg({
        # Core stats
        'kills': 'mean',
        'deaths': 'mean',
        'assists': 'mean',
        'matchId': 'count', # Count number of games played per champion
        
        # Combat stats
        'totalDamageDealtToChampions': 'mean',
        'totalDamageTaken': 'mean',
        'magicDamageDealtToChampions': 'mean',
        'physicalDamageDealtToChampions': 'mean',
        'trueDamageDealtToChampions': 'mean',
        'largestCriticalStrike': 'mean',
        'timeCCingOthers': 'mean',
        'totalHealsOnTeammates': 'mean',
        'totalDamageShieldedOnTeammates': 'mean',
        'damageSelfMitigated': 'mean',
        
        # Experience stats
        'champExperience': 'mean',
        'champLevel': 'mean', 
        
        # Economy stats
        'goldEarned': 'mean',
        'goldSpent': 'mean',
        'totalMinionsKilled': 'mean',
        'neutralMinionsKilled': 'mean',
        'bountyLevel': 'mean',
        'consumablesPurchased': 'mean',
        'itemsPurchased': 'mean', # int of number of items purchased in game, might be useless as it seems to count items and components as well
        
        # Vision stats
        'visionScore': 'mean',
        'wardsPlaced': 'mean',
        'wardsKilled': 'mean',
        'visionWardsBoughtInGame': 'mean',
        'detectorWardsPlaced': 'mean',
        
        # Team objectives
        'objectives_baron_kills': 'mean',
        'objectives_champion_kills': 'mean',
        'objectives_dragon_kills': 'mean',
        'objectives_inhibitor_kills': 'mean',
        'objectives_riftHerald_kills': 'mean',
        'objectives_tower_kills': 'mean',
        'damageDealtToBuildings': 'mean',
        'damageDealtToTurrets': 'mean',
        'damageDealtToObjectives': 'mean',
        
        # First objective rates
        'objectives_baron_first': 'mean',
        'objectives_dragon_first': 'mean',
        'objectives_inhibitor_first': 'mean',
        'objectives_riftHerald_first': 'mean',
        'objectives_tower_first': 'mean',
   
        # Game length related
        'timePlayed': 'mean',
        'longestTimeSpentLiving': 'mean',
        'gameEndedInEarlySurrender': 'mean', # Probably needs to be a derived stat, might need count first (might work with mean)
        'gameEndedInSurrender': 'mean', # Probably needs to be a derived stat, might need count first
        
        # Multikill stats
        'doubleKills': 'mean',
        'tripleKills': 'mean',
        'quadraKills': 'mean',
        'pentaKills': 'mean',
        
        # Items - find the dictionary with item ids and save it as a file (look what is the best way to store a dict as a file)

        # Combat achievements
        'firstBloodKill': 'mean',
        'firstBloodAssist': 'mean',
        'firstTowerKill': 'mean',
        'firstTowerAssist': 'mean',
        'turretKills': 'mean',
        'inhibitorKills': 'mean',
        'inhibitorTakedowns': 'mean', # Takedown = kill or assist
        'inhibitorsLost': 'mean', # Same number for all players in same team
        
        # Position/Role - find aggregation key word for most common string
        'individualPosition': lambda x: x.mode()[0] if not x.mode().empty else None, # Best guess for which position the player actually played in isolation of anything else
        'lane': lambda x: x.mode()[0] if not x.mode().empty else None # Gives slightly different string than above, might have something to do with where champ spent most of the time


    }).round(2)
    
    # Rename columns for clarity
    champion_stats = champion_stats.rename(columns={
        'matchId': 'games_played', # Rename the count column to a more descriptive name
        'objectives_baron_first': 'first_baron_rate',
        'objectives_dragon_first': 'first_dragon_rate',
        'objectives_inhibitor_first': 'first_inhibitor_rate',
        'objectives_riftHerald_first': 'first_herald_rate',
        'objectives_tower_first': 'first_tower_rate',
        'firstBloodKill': 'first_blood_rate',
        'firstTowerKill': 'first_tower_kill_rate'
    })
    
    # Calculate derived stats
    champion_stats['kda'] = ((champion_stats['kills'] + champion_stats['assists']) / 
                            champion_stats['deaths']).round(2)
    champion_stats['cs_per_minute'] = ((champion_stats['totalMinionsKilled'] + champion_stats['neutralMinionsKilled']) / 
                                      (champion_stats['timePlayed'] / 60)).round(2)
    champion_stats['gold_per_minute'] = (champion_stats['goldEarned'] / 
                                        (champion_stats['timePlayed'] / 60)).round(2)
    champion_stats['damage_per_minute'] = (champion_stats['totalDamageDealtToChampions'] / 
                                          (champion_stats['timePlayed'] / 60)).round(2)
    
    return champion_stats

# Create and display the aggregated data
champion_stats = aggregate_champion_data(df_participants, df_teams)
display(champion_stats.sort_values('games_played', ascending=False))
total_games = champion_stats['games_played'].sum()

print(total_games)

,,kills,deaths,assists,games_played,totalDamageDealtToChampions,totalDamageTaken,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,largestCriticalStrike,timeCCingOthers,totalHealsOnTeammates,totalDamageShieldedOnTeammates,damageSelfMitigated,champExperience,champLevel,goldEarned,goldSpent,totalMinionsKilled,neutralMinionsKilled,bountyLevel,consumablesPurchased,itemsPurchased,visionScore,wardsPlaced,wardsKilled,visionWardsBoughtInGame,detectorWardsPlaced,objectives_baron_kills,objectives_champion_kills,objectives_dragon_kills,objectives_inhibitor_kills,objectives_riftHerald_kills,objectives_tower_kills,damageDealtToBuildings,damageDealtToTurrets,damageDealtToObjectives,first_baron_rate,first_dragon_rate,first_inhibitor_rate,first_herald_rate,first_tower_rate,timePlayed,longestTimeSpentLiving,gameEndedInEarlySurrender,gameEndedInSurrender,doubleKills,tripleKills,quadraKills,pentaKills,first_blood_rate,firstBloodAssist,first_tower_kill_rate,firstTowerAssist,turretKills,inhibitorKills,inhibitorTakedowns,inhibitorsLost,individualPosition,lane,kda,cs_per_minute,gold_per_minute,damage_per_minute
championId,championName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
222,Jinx,6.50,5.67,6.70,1306,21919.09,19014.83,723.44,20763.69,431.10,675.36,16.81,4.07,0.00,10319.32,12646.38,13.99,11739.00,10699.64,195.53,8.53,6.50,2.05,19.20,14.63,7.50,2.58,0.60,0.51,0.32,26.84,1.75,0.84,0.44,5.60,5466.78,5466.78,14873.97,0.26,0.46,0.42,0.44,0.47,1646.89,562.38,0.01,0.29,0.97,0.17,0.03,0.01,0.08,0.09,0.13,0.05,1.75,0.28,0.51,0.83,BOTTOM,BOTTOM,2.33,7.43,427.68,798.56
81,Ezreal,6.27,5.00,6.81,1139,26130.79,21614.24,7806.15,17728.98,594.81,11.90,3.14,3.11,0.00,11997.58,13135.06,14.23,11637.01,10669.01,195.54,4.33,6.27,1.99,20.09,13.89,7.71,2.09,0.59,0.52,0.37,27.21,1.90,0.89,0.45,5.62,4923.31,4923.31,13503.66,0.30,0.48,0.41,0.45,0.53,1680.49,598.93,0.02,0.31,0.71,0.08,0.01,0.00,0.16,0.09,0.13,0.06,1.53,0.24,0.51,0.84,BOTTOM,BOTTOM,2.62,7.14,415.49,932.97
117,Lulu,1.39,4.54,14.24,1087,7428.21,15545.17,6113.45,1032.87,281.24,0.26,30.99,1218.80,5992.68,8736.08,10349.21,12.53,7603.68,6795.65,26.05,0.41,1.39,6.51,21.68,59.15,27.32,5.53,4.18,3.68,0.32,26.21,1.82,0.81,0.46,5.48,956.90,956.90,3664.21,0.27,0.49,0.40,0.46,0.49,1643.64,626.07,0.01,0.30,0.06,0.00,0.00,0.00,0.06,0.15,0.02,0.14,0.36,0.08,0.54,0.85,UTILITY,BOTTOM,3.44,0.97,277.57,271.16
131,Diana,6.47,5.61,7.50,936,20798.07,29799.58,17132.32,2588.69,1076.07,0.29,10.52,0.64,0.00,39320.66,14626.41,15.02,11900.51,10942.37,53.08,144.23,6.47,2.68,19.51,20.96,4.80,3.23,1.78,1.57,0.37,26.92,1.98,0.92,0.52,5.81,3034.20,3034.20,33378.64,0.32,0.54,0.44,0.52,0.51,1637.92,564.55,0.01,0.31,0.59,0.08,0.01,0.00,0.10,0.07,0.08,0.05,1.10,0.21,0.45,0.76,JUNGLE,JUNGLE,2.49,7.23,435.94,761.87
43,Karma,2.27,5.05,12.49,907,13755.80,18473.62,12144.49,1058.56,552.06,0.01,29.37,964.99,5254.94,12763.96,10889.08,12.81,8281.06,7472.91,57.65,0.73,2.27,6.11,21.47,55.19,24.97,5.68,3.70,3.29,0.33,26.63,1.84,0.83,0.44,5.44,1381.79,1381.79,3578.14,0.27,0.51,0.41,0.44,0.52,1650.87,579.50,0.01,0.33,0.08,0.00,0.00,0.00,0.05,0.16,0.04,0.11,0.46,0.11,0.50,0.83,UTILITY,BOTTOM,2.92,2.12,300.97,499.95
800,Mel,7.66,5.00,5.71,897,23805.32,17712.11,21913.99,1640.30,250.28,75.64,42.87,77.65,0.13,9240.96,13861.75,14.64,11454.44,10562.05,177.75,2.80,7.66,3.75,21.03,21.80,11.22,2.16,1.28,1.15,0.33,27.71,1.84,0.78,0.45,5.34,2397.58,2397.58,5623.63,0.27,0.47,0.38,0.45,0.47,1679.35,602.20,0.02,0.30,0.77,0.11,0.01,0.00,0.11,0.07,0.05,0.03,0.76,0.12,0.42,0.89,MIDDLE,MIDDLE,2.67,6.45,409.25,850.52
267,Nami,1.50,4.67,14.95,849,9220.29,15986.33,7955.06,1084.61,180.04,0.00,30.29,9259.60,1696.10,7244.91,10447.54,12.57,7669.33,6855.31,22.60,0.19,1.50,6.32,21.96,60.28,27.51,5.59,4.03,3.50,0.32,26.54,1.83,0.88,0.41,5.51,1226.93,1226.93,2732.52,0.27,0.48,0.42,0.41,0.48,1659.54,614.29,0.01,0.29,0.05,0.00,0.00,0.00,0.04,0.17,0.03,0.14,0.45,0.10,0.58,0.88,UTILITY,BOTTOM,3.52,0.82,277.28,333.36
145,Kaisa,7.31,5.67

59980


In [24]:
champion_stats.to_csv("champion_stats.csv", index = False)

For Debugging

In [14]:
def aggregate_champion_data(df_participants: pd.DataFrame, df_teams: pd.DataFrame) -> pd.DataFrame:
    # Create a teamId column in participants based on participantId (1-5 = 100, 6-10 = 200)
    df_participants['teamId'] = df_participants['participantId'].apply(lambda x: 100 if x <= 5 else 200)
    
    # Merge participants with their team data
    merged_df = df_participants.merge(
        df_teams,
        on=['matchId', 'teamId'],
        how='left'
    )
    
    # Aggregate by specific champion for debugging
    champion_stats = merged_df[merged_df['championName'] == 'Shyvana']
    return champion_stats
        

# Create and display the aggregated data
champion_stats = aggregate_champion_data(df_participants, df_teams)
display(champion_stats)

,PlayerScore0,PlayerScore1,PlayerScore10,PlayerScore11,PlayerScore2,PlayerScore3,PlayerScore4,PlayerScore5,PlayerScore6,PlayerScore7,PlayerScore8,PlayerScore9,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,placement,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerAugment5,playerAugment6,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,retreatPings,riotIdGameName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win_x,challenges_12AssistStreakCount,challenges_HealFromMapSources,challenges_InfernalScalePickup,challenges_SWARM_DefeatAatrox,challenges_SWARM_DefeatBriar,challenges_SWARM_DefeatMiniBosses,challenges_SWARM_EvolveWeapon,challenges_SWARM_Have3Passives,challenges_SWARM_KillEnemy,challenges_SWARM_PickupGold,challenges_SWARM_ReachLevel50,challenges_SWARM_Survive15Min,challenges_SWARM_WinWith5EvolvedWeapons,challenges_abilityUses,challenges_acesBefore15Minutes,challenges_alliedJungleMonsterKills,challenges_baronTakedowns,challenges_blastConeOppositeOpponentCount,challenges_bountyGold,challenges_buffsStolen,challenges_completeSupportQuestInTime,challenges_controlWardsPlaced,challenges_damagePerMinute,challenges_damageTakenOnTeamPercentage,challenges_dancedWithRiftHerald,challenges_deathsByEnemyChamps,challenges_dodgeSkillShotsSmallWindow,challenges_doubleAces,challenges_dragonTakedowns,challenges_earlyLaningPhaseGoldExpAdvantage,challenges_effectiveHealAndShielding,challenges_elderDragonKillsWithOpposingSoul,challenges_elderDragonMultikills,challenges_enemyChampionImmobilizations,challenges_enemyJungleMonsterKills,challenges_epicMonsterKillsNearEnemyJungler,challenges_epicMonsterKillsWithin30SecondsOfSpawn,challenges_epicMonsterSteals,challenges_epicMonsterStolenWithoutSmite,challenges_firstTurretKilled,challenges_fistBumpParticipation,challenges_flawlessAces,challenges_fullTeamTakedown,challenges_gameLength,challenges_getTakedownsInAllLanesEarlyJungleAsLaner,challenges_goldPerMinute,challenges_hadOpenNexus,challenges_immobilizeAndKillWithAlly,challenges_initialBuffCount,challenges_initialCrabCount,challenges_jungleCsBefore10Minutes,challenges_junglerTakedownsNearDamagedEpicMonster,challenges_kTurretsDestroyedBeforePlatesFall,challenges_kda,challenges_killAfterHiddenWithAlly,challenges_killParticipation,challenges_killedChampTookFullTeamDamageSurvived,ch